In [ ]:
# Cell 1: Simple Setup for LSTM (CPU-friendly)
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Setup paths
repo_root = Path.cwd()
for up in [repo_root, repo_root.parent, repo_root.parent.parent]:
    if (up / "src").exists():
        repo_root = up
        break

import sys
sys.path.insert(0, str(repo_root))

# Simple TensorFlow check
try:
    import tensorflow as tf
    print(f"✓ TensorFlow version: {tf.__version__}")
    
    # Disable GPU if causing issues
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'  # Force CPU usage
    print("✓ Using CPU (GPU disabled for stability)")
    
except ImportError:
    print("ERROR: TensorFlow not installed!")
    print("For Mac: pip install tensorflow-macos")
    print("For others: pip install tensorflow")
    raise

# Load data
DATA = repo_root / "data" / "processed" / "flights_nativecadence_enu_kinematics.parquet"
df = pd.read_parquet(DATA)

print(f"\n✓ Loaded {len(df):,} rows")
print(f"✓ Continuous runs: {df.groupby(['flight_id', 'run_id']).ngroups}")

# Check EKF baseline
import pickle
ekf_path = repo_root / "results" / "ekf_baseline.pkl"
if ekf_path.exists():
    with open(ekf_path, 'rb') as f:
        ekf_results = pickle.load(f)
    print(f"\n✓ EKF Baseline: {ekf_results['summary']['mean_rmse_60s']:.1f}m RMSE at 60s")
    print("  Goal: Beat this with LSTM!")